In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import zipfile
from keras.metrics import Precision, Recall


In [ ]:
with zipfile.ZipFile("/content/Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("Dataset")

In [ ]:
test_data = image_dataset_from_directory("/content/Dataset/Dataset/Test",image_size = (224,224),batch_size = 128*2)


Found 80362 files belonging to 2 classes.


In [ ]:
train_data = image_dataset_from_directory("/content/Dataset/Dataset/Train",image_size = (224,224),batch_size = 50)


Found 120542 files belonging to 2 classes.


In [ ]:
conv_base = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet'
)
conv_base.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
inputs = keras.Input(shape = (224,224,3))
x = keras.applications.resnet50.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256,activation = "relu")(x)
x = layers.Dense(128,activation = "relu")(x)
x  =layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model  = keras.Model(inputs = inputs,outputs = outputs)

In [ ]:
model.compile(optimizer="adam" ,loss ="binary_crossentropy",metrics =["accuracy", Precision(name='precision'), Recall(name='recall')])

In [ ]:
model.fit(train_data,epochs = 5)


Epoch 1/5
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 177s 69ms/step - accuracy: 0.8528 - loss: 0.3797 - precision: 0.6837 - recall: 0.5637
Epoch 2/5
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 161s 67ms/step - accuracy: 0.8719 - loss: 0.3310 - precision: 0.7616 - recall: 0.5569
Epoch 3/5
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 161s 67ms/step - accuracy: 0.8726 - loss: 0.3282 - precision: 0.7663 - recall: 0.5556
Epoch 4/5
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 161s 67ms/step - accuracy: 0.8724 - loss: 0.3271 - precision: 0.7624 - recall: 0.5599
Epoch 5/5
2411/2411 ━━━━━━━━━━━━━━━━━━━━ 161s 67ms/step - accuracy: 0.8739 - loss: 0.3246 - precision: 0.7682 - recall: 0.5622


In [ ]:
all_labels = []
for i,l in test_data:
  all_labels.extend(l.numpy())

In [ ]:
clean_labels = [int(x) for x in all_labels]

In [ ]:
y_pred = model.predict(test_data)

314/314 ━━━━━━━━━━━━━━━━━━━━ 140s 390ms/step


In [ ]:
y_pred_labels = (y_pred > 0.5).astype(int).flatten()


In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score

In [ ]:
accuracy_score(clean_labels,y_pred_labels)

0.6850999228491078

In [ ]:
recall_score(clean_labels,y_pred_labels)

0.18654306220095693

In [ ]:
precision_score(clean_labels,y_pred_labels)

0.2104020507285483

In [ ]:
f1_score(clean_labels,y_pred_labels)

0.19775551610448897

In [ ]:
confusion_matrix(clean_labels,y_pred_labels)

array([[51937, 11705],
       [13601,  3119]])